In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np
from six.moves import xrange
import tensorflow as tf
from tensorflow.python.platform import flags
import time 
import logging
import os
from cleverhans.attacks import CarliniWagnerL2
from cleverhans.utils import pair_visual, grid_visual, AccuracyReport
from cleverhans.utils import set_log_level
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import model_train, model_eval, tf_model_load
from cleverhans_tutorials.tutorial_models import make_basic_cnn


ModuleNotFoundError: No module named 'keras'

In [ ]:



def mnist_cw(train_start=0, train_end=60000, test_start=0,
                      test_end=10000, nb_epochs=6,
                      batch_size=128, nb_classes=10, source_samples=10,
                      learning_rate=0.001, attack_iterations=10,
                      model_path=os.path.join("models", "mnist"),
                      targeted=True):
    
    """
    reproducing C & W attack on MNIST data set.
    :param train_start: index of first training set example
    :param train_end: index of last training set example
    :param test_start: index of first test set example
    :param test_end: index of last test set example
    :param nb_epochs: number of epochs to train model
    :param batch_size: size of training batches
    :param nb_classes: number of output classes
    :param source_samples: number of test inputs to attack
    :param learning_rate: learning rate for training
    :param model_path: path to the model file
    :param targeted: should we run a targeted attack? or untargeted?
    :return: an AccuracyReport object
    """
    # Object used to keep track of (and return) key accuracies
    report = AccuracyReport()

    # Set TF random seed to improve reproducibility
    tf.set_random_seed(1234)

    # Create TF session
    sess = tf.Session()
    print("Created TensorFlow session.")

    set_log_level(logging.DEBUG)

    # Get MNIST test data
    X_train, Y_train, X_test, Y_test = data_mnist(train_start=train_start,
                                                  train_end=train_end,
                                                  test_start=test_start,
                                                  test_end=test_end)
    # MNIST-specific dimensions
    img_rows = 28
    img_cols = 28
    channels = 1
    # Define input TF placeholder
    x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols, channels))
    y = tf.placeholder(tf.float32, shape=(None, nb_classes))

    # Define TF model graph
    model = make_basic_cnn()
    preds = model(x)
    print("Defined TensorFlow model graph.")

    # Train an MNIST model
    train_params = {
        'nb_epochs': nb_epochs,
        'batch_size': batch_size,
        'learning_rate': learning_rate,
        'train_dir': os.path.join(*os.path.split(model_path)[:-1]),
        'filename': os.path.split(model_path)[-1]
    }

    rng = np.random.RandomState([2017, 12, 10])
    
    model_train(sess, x, y, preds, X_train, Y_train, args=train_params,
                save=os.path.exists("models"), rng=rng)
    start = time.time()
    
    
    # Instantiating C & W attack
    cw = CarliniWagnerL2(model, back='tf', sess=sess)

    if targeted:
        adv_inputs = np.array(
            [[instance] * nb_classes for
             instance in X_test[:source_samples]], dtype=np.float32)
        one_hot = np.zeros((nb_classes, nb_classes))
        one_hot[np.arange(nb_classes), np.arange(nb_classes)] = 1

        adv_inputs = adv_inputs.reshape(
            (source_samples * nb_classes, img_rows, img_cols, 1))
        adv_ys = np.array([one_hot] * source_samples,
                          dtype=np.float32).reshape((source_samples *
                                                     nb_classes, nb_classes))
        yname = "y_target"
    else:
        adv_inputs = X_test[:source_samples]

        adv_ys = None
        yname = "y"

    cw_params = {'binary_search_steps': 1,
                 yname: adv_ys,
                 'max_iterations': attack_iterations,
                 'learning_rate': 0.1,
                 'batch_size': source_samples * nb_classes if
                 targeted else source_samples,
                 'initial_const': 10}

    adv = cw.generate_np(adv_inputs,
                         **cw_params)

    eval_params = {'batch_size': np.minimum(nb_classes, source_samples)}
    if targeted:
        adv_accuracy = model_eval(
            sess, x, y, preds, adv, adv_ys, args=eval_params)
    else:
        adv_accuracy = 1 - \
            model_eval(sess, x, y, preds, adv, Y_test[
                       :source_samples], args=eval_params)

    end = = time.time()
    print('--------------------------------------')

    # Compute evaluation metrics
    print('Acc on adversarial examples. {0:.4f}'.format(adv_accuracy))
    report.clean_train_adv_eval = 1. - adv_accuracy

    percent_perturbed = np.mean(np.sum((adv - adv_inputs)**2,
                                       axis=(1, 2, 3))**.5)/784
    print('Mean perturbations score {0:.4f}'.format(percent_perturbed))
    print('took %d seconds'%(end-start))
    # Close TF session
    sess.close()


tf.app.run()
try:
    modelpath = os.path.join("models", "mnist")
except:
    modelpath = None
for nb_epochs in [6,12,18]:
    mnist_tutorial_cw(nb_epochs=nb_epochs,
                          batch_size=128,
                          nb_classes=10,
                          source_samples=10,
                          learning_rate=0.01,
                          attack_iterations=100,
                          model_path=modelpath,
                          targeted=True)
    
for attack_iter in [10,50,80,100]:
    mnist_tutorial_cw(nb_epochs=nb_epochs,
                          batch_size=128,
                          nb_classes=10,
                          source_samples=10,
                          learning_rate=0.01,
                          attack_iterations=attack_iter,
                          model_path=modelpath,
                          targeted=True)
    

